### Install alibi_detect library

In [1]:
import numpy as np
np.__version__

'1.26.4'

In [2]:
!pip install alibi alibi_detect


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.1/522.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.5/381.5 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully 

In [1]:
# importing all necessary libraries
import alibi
from alibi_detect.cd import ChiSquareDrift, TabularDrift
from alibi_detect.saving import save_detector, load_detector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

In [3]:
# loading train and production data from the github
# this is splitted in first notebook
train_url = "https://raw.githubusercontent.com/ritikisb83/CTAssignmentgroup5/refs/heads/main/Datasets/restaurant_train_df.parquet"
train_df = pd.read_parquet(train_url, engine="pyarrow")
prod_url = "https://raw.githubusercontent.com/ritikisb83/CTAssignmentgroup5/refs/heads/main/Datasets/restaurant_prod_df.parquet"
prod_df = pd.read_parquet(prod_url, engine="pyarrow")

In [4]:
# creating features set
features_set = ['Number_of_Customers', 'Menu_Price', 'Marketing_Spend', 'Cuisine_Type',
       'Average_Customer_Spending', 'Promotions', 'Reviews',
       'Monthly_Revenue']

In [5]:
# Creating X features
x_columns = ['Number_of_Customers', 'Menu_Price', 'Marketing_Spend', 'Cuisine_Type',
       'Average_Customer_Spending', 'Promotions', 'Reviews']

In [6]:
# creating categorical features
cat_vars = ['Cuisine_Type']

In [7]:
# Creating numerical features
num_vars = list(set(x_columns) - set(cat_vars))

In [8]:
# Splitting train & prod dataset into X & y
x_train = train_df[x_columns]
y_train = train_df['Monthly_Revenue']
x_prod = prod_df[x_columns]
y_prod = prod_df['Monthly_Revenue']

### Measure the drift

In [11]:
cat_vars = [3]
categories_per_feature = {f: None for f in cat_vars}
categories_per_feature

{3: None}

In [12]:
cd = TabularDrift(x_train.values,
                  p_val=.05,
                  categories_per_feature=categories_per_feature)

In [13]:
filepath = 'restaurantdrift'  # change to directory where detector is saved
save_detector(cd, filepath, legacy = True)

In [14]:
cd = load_detector(filepath)

In [15]:
preds = cd.predict(x_prod.to_numpy())

### Printing the test results

- KS test for the numerical features
- chi-squared test for the categorical features

In [16]:
# Iterate through the range of features
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = x_columns[f] # Get the feature name
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')  # Print the feature name, statistical test, statistic value, and p-value

Number_of_Customers -- K-S 0.045 -- p-value 0.908
Menu_Price -- K-S 0.058 -- p-value 0.665
Marketing_Spend -- K-S 0.085 -- p-value 0.216
Cuisine_Type -- Chi2 0.743 -- p-value 0.863
Average_Customer_Spending -- K-S 0.068 -- p-value 0.465
Promotions -- K-S 0.015 -- p-value 1.000
Reviews -- K-S 0.107 -- p-value 0.061
